In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=a2cbd081a1a6e68e40e566f971f4f3ab79722586be9a2fa85b34307fb82cb92e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
!ls -ltr

total 4
drwxr-xr-x 1 root root 4096 Sep 14 13:23 sample_data


In [ ]:
pwd

'/content'

In [ ]:
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv

--2023-09-17 21:47:43--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv [following]
--2023-09-17 21:47:43--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22608 (22K) [text/csv]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  22.08K  --.-KB/s    in 0s      

2023-09-17 21:47:43 (118 MB/s) - ‘cars.csv’ saved [22608/22608]



In [ ]:
!cat cars.csv

In [ ]:
df=spark.read.csv("cars.csv",sep=";", header="True")
df.show(5)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows



In [ ]:
!wget https://raw.githubusercontent.com/pathwaytipsorg/gcp-data-engineering/master/dproc/data/customer_data.csv

--2023-09-17 22:01:33--  https://raw.githubusercontent.com/pathwaytipsorg/gcp-data-engineering/master/dproc/data/customer_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4319 (4.2K) [text/plain]
Saving to: ‘customer_data.csv’

customer_data.csv   100%[===================>]   4.22K  --.-KB/s    in 0s      

2023-09-17 22:01:33 (15.5 MB/s) - ‘customer_data.csv’ saved [4319/4319]



In [ ]:
!cat customer_data.csv

In [ ]:
df = spark.read.csv("customer_data.csv",header=True,inferSchema=True)
df.show(6)

+-----------+----------+---------+--------------------+------------+----------+------+-------------+-------------+
|customer_id|first_name|last_name|               email|phone_number|       dob|gender|     location|date_of_entry|
+-----------+----------+---------+--------------------+------------+----------+------+-------------+-------------+
|          1|      Jane|      Doe|jane.doe@example.com|    555-1234|1990-01-01|Female|     New York|   2022-01-01|
|          1|      Jane|      Doe|jane.doe@example.com|    555-1234|1990-01-01|Female|       Boston|   2022-02-01|
|          1|      Jane|      Doe|jane.doe@example.com|    555-5678|1990-01-01|Female|       Boston|   2022-03-01|
|          2|      John|      Doe|john.doe@example.com|    555-4321|1991-02-02|  Male|San Francisco|   2022-04-01|
|          2|      John|      Doe|john.doe@example.com|    555-4321|1991-02-02|  Male|San Francisco|   2022-05-01|
|          2|      John|      Doe|john.doe@example.com|    555-8765|1991-02-02| 

In [ ]:
df.select("first_name").show()

In [ ]:
#
from pyspark.sql.functions import col
#df.select(col("first_name")).show()
#df.orderBy(col("first_name").desc()).show()
df.orderBy(col('date_of_entry').desc()).dropDuplicates(['customer_id']).show()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number

Wpart=Window.partitionBy('customer_id').orderBy(col('date_of_entry').desc())
df_step1=df.withColumn('row_id',row_number().over(Wpart))
df_step2=df_step1.filter(df_step1['row_id']==1)
df_step2.show()

+-----------+----------+----------+--------------------+--------------------+----------+------+--------------------+-------------+------+
|customer_id|first_name| last_name|               email|        phone_number|       dob|gender|            location|date_of_entry|row_id|
+-----------+----------+----------+--------------------+--------------------+----------+------+--------------------+-------------+------+
|          1|      Jane|       Doe|jane.doe@example.com|            555-5678|1990-01-01|Female|              Boston|   2022-03-01|     1|
|          2|      John|       Doe|john.doe@example.com|            555-8765|1991-02-02|  Male|       San Francisco|   2022-06-01|     1|
|          3|   Michael|     Jones| pmendez@example.org|        883.178.1278|1985-09-14|  Male|315 Jensen Square...|   2023-01-04|     1|
|          4|     Marie|   Robbins| david47@example.com| +1-609-590-0742x350|1976-03-30|Female|44816 Stewart Pin...|   2022-07-20|     1|
|          5|     Susan|Richardson

In [ ]:
from pyspark.sql.functions import col
data = [("Product A", 10, 50),
        ("Product B", 15, 75),
        ("Product A", 5, 60),
        ("Product C", 8, 90),
        ("Product B", 20, 100)]
columns = ["product", "quantity", "price"]

df = spark.createDataFrame(data,columns)

# view & sql : assignment
df = df.withColumn("revenue", col("quantity") * col("price"))
df.show()

#df = df.groupBy("product").agg(sum("revenue").alias("total_revenue"))
#df.show()

df = df.orderBy(col("total_revenue").desc()).first()

print("Top selling product: ",df["product"])
print("Total revenue: ",df["total_revenue"])"""